<a href="https://colab.research.google.com/github/AnaJlia999/gsi073/blob/main/GSI073_aula0_redes_neurais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GSI073 - Tópicos Especiais de Inteligência Artificial

## Definição dos dados

In [ ]:
import torch; import sklearn; from torch import nn
from sklearn.model_selection import train_test_split

# 1. Carregar dados
iris = sklearn.datasets.load_iris()
X = iris.data        # 4 features: sépalas e pétalas
y = (iris.target == 1).astype(float)  # 1 se Versicolor, 0 caso contrário

# 2. Dividir os dados em 80 amostras para treinamento e 70 para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# 2. Preparar dados para pytorch
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)

X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)

##Definição do modelo e treinamento

In [ ]:
# 3. Definir modelo
import torch.nn.functional as F
class RedeNeural(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(RedeNeural, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [ ]:
# Criar modelo
modelo = RedeNeural(4, 8, 1)  # 4 features → 1 saída (probabilidade de ser Versicolor)
# O 8 é o hidden_dim. O hidden_dim define quantos neurônios existem na camada escondida.

import copy
modelo_clonado = copy.deepcopy(modelo)

learning_rate = 0.1

# Definir função de perda e algoritmo de otimização
funcao_perda = torch.nn.BCEWithLogitsLoss()  # combinação de sigmoid + BCE
optimizer = torch.optim.SGD(modelo.parameters(), lr=learning_rate)

## Execução do treinamento com optimizer SGD

In [ ]:
# Loop de treino
for epoch in range(1000):
    optimizer.zero_grad()           # Limpa gradientes
    outputs = modelo(X_train)             # Forward
    loss = funcao_perda(outputs, y_train) # Calcula perda
    loss.backward()                 # Calcula derivadas do gradiente
    optimizer.step()                # Aplica regra de alteração dos parâmetros

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/100], Loss: {loss.item():.4f}")

# Treino com regra de gradiente descendente manual

In [ ]:
for epoch in range(1000):
    optimizer.zero_grad()           # Limpa gradientes
    outputs = modelo_clonado(X)     # Forward
    loss = funcao_perda(outputs, y) # Calcula perda
    loss.backward()                 # Calcula derivadas do gradiente

    with torch.no_grad():
        for param in modelo_clonado.parameters():
            param -= learning_rate * param.grad  # regra de atualização de pesos

    if (epoch + 1) % 100 == 0:
        print(f"Época [{epoch+1}/1000], Loss: {loss.item():.4f}")

##Teste do modelo

In [ ]:
from sklearn.metrics import accuracy_score
with torch.no_grad():
    outputs_test = modelo(X_test)
    previsoes = torch.sigmoid(modelo(X_test))
    previsoes_classes = (previsoes > 0.5).float()
    acuracia = accuracy_score(y_test,previsoes_classes)
    print(f"Acurácia do modelo: {acuracia :.4f}")


##hidden_dim

In [ ]:
hidden_dims = [2, 4, 8, 16, 32, 64]
resultados = {}

for h in hidden_dims:
    modelo_temp = RedeNeural(4, h, 1)
    optimizer_temp = torch.optim.SGD(modelo_temp.parameters(), lr=0.1)
    funcao_perda = torch.nn.BCEWithLogitsLoss()

    for epoch in range(100):
        optimizer_temp.zero_grad()
        outputs = modelo_temp(X_train)
        loss = funcao_perda(outputs, y_train)
        loss.backward()
        optimizer_temp.step()

    resultados[h] = loss.item()

print("Loss na época 100 para diferentes tamanhos de camada escondida:")
for h, l in resultados.items():
    print(f"hidden_dim={h}: loss={l:.4f}")